In [155]:
import sys
import random
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import sumolib
import traci
from sumolib import checkBinary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import sys
import io
from contextlib import redirect_stdout
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from utils import *


if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

sumoBinary = checkBinary('sumo-gui')
# sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
Using device: cuda


## Prepare the planned path

In [156]:
planned_path = get_planned_path()
path_values = list(planned_path.values())

 Retrying in 1 seconds
***Starting server on port 39105 ***
Loading net-file from './config/osm.net.xml.gz' ... done (81ms).
Loading additional-files from './config/osm.poly.xml.gz' ... done (229ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 10476.00
Reason: TraCI requested termination.
Performance: 
 Duration: 4.94s
 TraCI-Duration: 3.19s
 Real time factor: 2120.22
 UPS: 65263.711799
Vehicles: 
 Inserted: 1530
 Running: 0
 Waiting: 0
Statistics (avg of 1530):
 RouteLength: 1611.54
 Speed: 7.66
 Duration: 210.76
 WaitingTime: 12.69
 TimeLoss: 40.04
 DepartDelay: 0.55



## Prepare the dataset

In [157]:
# Example DataFrame loading
df = pd.read_csv('edinburgh_trajectories.csv')
num_cols = df.shape[1]
position_indices = [i for i in range(num_cols) if i % 4 == 1 or i % 4 == 2]
position_df = df.iloc[:, position_indices]
position_array = position_df.to_numpy()
sequence_length = len(position_indices) // 2
tensor_list = []

for row in position_array:
    reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
    tensor_list.append(reshaped_tensor)

all_trajectories_tensor = torch.stack(tensor_list) / 10

def missing(x, y, z, p):
    return torch.tensor(np.repeat(np.random.rand(x * y) < p, z).reshape(x, y, z)).float()

def generate_masks(tensors, min_mask_ratio=0.2, max_mask_ratio=0.4, missing_ratio=0.5, complete_traj_ratio=0.8):
    initial_masks = missing(tensors.shape[0], tensors.shape[1], tensors.shape[2], missing_ratio)
    masks = []
    for initial_mask in initial_masks:
        if np.random.rand() < complete_traj_ratio:
            masks.append(torch.zeros_like(initial_mask).tolist())
            continue
        seq_length = initial_mask.shape[0]
        mask_start = np.random.randint(int(seq_length * min_mask_ratio), int(seq_length * max_mask_ratio))
        mask = torch.zeros_like(initial_mask)
        mask[:, :mask_start] = 1
        mask = initial_mask * mask
        mask[0] = 0
        mask[1] = 0
        masks.append(mask.tolist())
    return torch.tensor(masks)

# split the data into training and validation sets
# because the data is randomly generated, we don't need to shuffle it
train_ratio = 0.8
train_size = int(train_ratio * all_trajectories_tensor.shape[0])
train_trajectories_tensor = all_trajectories_tensor[:train_size]
val_trajectories_tensor = all_trajectories_tensor[train_size:]

train_path_values = path_values[:train_size]
val_path_values = path_values[train_size:]

train_mask = generate_masks(train_trajectories_tensor)

class myDataset(Dataset):
    def __init__(self, tensor, input_mask):
        self.tensor = tensor.float().to(device)
        self.input_mask = input_mask.float().to(device)
    def __len__(self):
        return len(self.tensor)
    def __getitem__(self, idx):
        return self.tensor[idx], self.input_mask[idx]

train_dataset = myDataset(train_trajectories_tensor, train_mask)

In [120]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, int(hidden_size/2))
        self.fc2 = nn.Linear(int(hidden_size/2), num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x, h0=None):
        if h0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, hidden = self.gru(x, h0)  
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out, hidden

class CustomMSE(nn.Module):
    def __init__(self):
        super(CustomMSE, self).__init__()

    def forward(self, output, target):
        # where target is 0
        mask = (target != 0).float().to(device)
        mse = torch.mean(((output - target) ** 2) * mask)
        directional_diff = torch.mean(torch.abs(torch.atan2(output[:, 1], output[:, 0]) - torch.atan2(target[:, 1], target[:, 0])))
        return mse + directional_diff

## Training without masks

In [ ]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            for t in range(1, seq_len):
                prediction, hidden = model(current_input, hidden)

                previous_input = inputs[:, t-1, :]
                current_input = inputs[:, t, :].unsqueeze(1)
                loss += criterion(prediction - previous_input, inputs[:, t, :]-previous_input) #(current_input-previous_input).squeeze(1))
                
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=2, hidden_size=128, num_layers=2, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 50, optimizer, criterion)

Epoch 1, Loss: 1418839.8375
Epoch 2, Loss: 1317015.0625
Epoch 3, Loss: 1167506.846875
Epoch 4, Loss: 921565.25625
Epoch 5, Loss: 700849.5234375
Epoch 6, Loss: 534546.7859375
Epoch 7, Loss: 421176.6390625
Epoch 8, Loss: 346989.759375
Epoch 9, Loss: 262977.4330078125
Epoch 10, Loss: 193418.87421875
Epoch 11, Loss: 139608.171875
Epoch 12, Loss: 105166.70625
Epoch 13, Loss: 81678.2474609375
Epoch 14, Loss: 57730.5259765625
Epoch 15, Loss: 40922.224609375
Epoch 16, Loss: 28032.987939453124
Epoch 17, Loss: 20098.87451171875
Epoch 18, Loss: 14365.5091796875
Epoch 19, Loss: 11668.78740234375
Epoch 20, Loss: 10230.59111328125
Epoch 21, Loss: 9555.001782226562
Epoch 22, Loss: 9219.741650390624
Epoch 23, Loss: 8741.28974609375
Epoch 24, Loss: 8825.73759765625
Epoch 25, Loss: 8873.706494140624
Epoch 26, Loss: 8916.641333007812
Epoch 27, Loss: 8772.619653320313
Epoch 28, Loss: 8680.322631835938
Epoch 29, Loss: 8535.723486328125
Epoch 30, Loss: 8604.791845703125
Epoch 31, Loss: 8663.745874023438
Epo

In [115]:
def checkModel(model, inputs, masks):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            prediction, hidden = model(current_input, hidden)
            current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            print(prediction[2], inputs[:, t, :][2])

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=1, complete_traj_ratio=0)
val_dataset = myDataset(val_trajectories_tensor, val_mask)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
inputs, masks = next(iter(val_dataloader))

checkModel(model, inputs, masks)

tensor([43.2485, 52.5396], device='cuda:0') tensor([43.0983, 49.1154], device='cuda:0')
tensor([44.9612, 48.3119], device='cuda:0') tensor([42.8314, 48.7820], device='cuda:0')
tensor([47.0588, 49.4371], device='cuda:0') tensor([42.4789, 48.3515], device='cuda:0')
tensor([48.0593, 49.0440], device='cuda:0') tensor([42.0778, 47.8618], device='cuda:0')
tensor([49.4910, 49.4640], device='cuda:0') tensor([41.7241, 47.4299], device='cuda:0')
tensor([50.8521, 50.2545], device='cuda:0') tensor([41.3507, 46.9740], device='cuda:0')
tensor([51.6722, 50.8181], device='cuda:0') tensor([40.9493, 46.4781], device='cuda:0')
tensor([52.1197, 51.1526], device='cuda:0') tensor([40.5827, 46.0237], device='cuda:0')
tensor([52.3842, 51.3498], device='cuda:0') tensor([40.3831, 45.8530], device='cuda:0')
tensor([52.5853, 51.4756], device='cuda:0') tensor([40.3436, 45.8406], device='cuda:0')
tensor([52.7685, 51.5643], device='cuda:0') tensor([40.3392, 45.8392], device='cuda:0')
tensor([52.9588, 51.6316], devic

In [125]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((prediction - inputs[:, t, :]) * 10, dim=1) * (inputs[:, t, :] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

for i in [0.9, 0.8, 0.7, 0.6, 0.5]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = myDataset(val_trajectories_tensor, val_mask)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 260.393017879635
Validation Loss: 137.36904396178304
Validation Loss: 80.10986029615066
Validation Loss: 48.81005383859175
Validation Loss: 35.16651754170379


## Training with masks

In [126]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            for t in range(1, seq_len):
                prediction, hidden = model(current_input, hidden)

                previous_input = inputs[:, t-1, :]
                if epoch > 30:
                    current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                else:
                    current_input = inputs[:, t, :].unsqueeze(1)
                loss += criterion(prediction - previous_input, inputs[:, t, :]-previous_input) #(current_input-previous_input).squeeze(1))
                
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=2, hidden_size=128, num_layers=2, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 50, optimizer, criterion)

Epoch 1, Loss: 1385423.53125
Epoch 2, Loss: 1164602.709375
Epoch 3, Loss: 838574.303125
Epoch 4, Loss: 536211.296875
Epoch 5, Loss: 351630.7921875
Epoch 6, Loss: 305161.88515625
Epoch 7, Loss: 234605.4828125
Epoch 8, Loss: 135569.37890625
Epoch 9, Loss: 92482.095703125
Epoch 10, Loss: 68134.6650390625
Epoch 11, Loss: 49702.3736328125
Epoch 12, Loss: 34424.21015625
Epoch 13, Loss: 25589.719775390626
Epoch 14, Loss: 20085.9224609375
Epoch 15, Loss: 15853.012158203124
Epoch 16, Loss: 12991.934326171875
Epoch 17, Loss: 11481.109423828126
Epoch 18, Loss: 10429.297143554688
Epoch 19, Loss: 10133.690966796876
Epoch 20, Loss: 9832.507202148438
Epoch 21, Loss: 9844.470263671876
Epoch 22, Loss: 9722.3091796875
Epoch 23, Loss: 9430.59912109375
Epoch 24, Loss: 9336.203076171874
Epoch 25, Loss: 9084.710913085937
Epoch 26, Loss: 9165.718090820312
Epoch 27, Loss: 9153.619970703125
Epoch 28, Loss: 9134.27490234375
Epoch 29, Loss: 9015.09287109375
Epoch 30, Loss: 9077.490307617187
Epoch 31, Loss: 9390.

In [129]:
def checkModel(model, inputs, masks):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            prediction, hidden = model(current_input, hidden)
            current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            print(prediction[2], inputs[:, t, :][2])

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=1, complete_traj_ratio=0)
val_dataset = myDataset(val_trajectories_tensor, val_mask)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
inputs, masks = next(iter(val_dataloader))

checkModel(model, inputs, masks)

tensor([111.2589,  87.2843], device='cuda:0') tensor([107.7088,  90.2773], device='cuda:0')
tensor([108.9298,  89.8355], device='cuda:0') tensor([107.8385,  89.9412], device='cuda:0')
tensor([108.5739,  89.4880], device='cuda:0') tensor([108.0062,  89.4524], device='cuda:0')
tensor([108.2597,  89.0661], device='cuda:0') tensor([108.2320,  88.7565], device='cuda:0')
tensor([108.0601,  88.5980], device='cuda:0') tensor([108.5044,  87.9346], device='cuda:0')
tensor([108.0975,  88.2589], device='cuda:0') tensor([108.8661,  87.3475], device='cuda:0')
tensor([108.2483,  87.9837], device='cuda:0') tensor([109.0687,  87.1390], device='cuda:0')
tensor([108.4727,  87.7447], device='cuda:0') tensor([109.1130,  87.0936], device='cuda:0')
tensor([108.7487,  87.5269], device='cuda:0') tensor([109.1273,  87.0788], device='cuda:0')
tensor([109.0484,  87.3213], device='cuda:0') tensor([109.1355,  87.0704], device='cuda:0')
tensor([109.3541,  87.1201], device='cuda:0') tensor([109.1369,  87.0690], devic

In [130]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((prediction - inputs[:, t, :]) * 10, dim=1) * (inputs[:, t, :] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

for i in [0.9, 0.8, 0.7, 0.6, 0.5]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = myDataset(val_trajectories_tensor, val_mask)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 86.42295743602347
Validation Loss: 42.63542210684771
Validation Loss: 28.914880207213297
Validation Loss: 22.28685226644343
Validation Loss: 17.99894012472735


## Experiment with TD

In [150]:
class GRU_td(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU_td, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, int(hidden_size/2))
        self.fc2 = nn.Linear(int(hidden_size/2), num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x, h0=None, temporal_lag=0):
        if h0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        temporal_features = 1 / torch.exp(temporal_lag).to(device)
        h0 = h0 * temporal_features
        out, hidden = self.gru(x, h0)
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)

        return out, hidden

def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1) # start with the first input step
            for t in range(1, seq_len):
                if t <= 1 :#or epoch <= 30:
                    temporal_lag = torch.ones(inputs.size(0), 1).to(device)
                else:
                    temporal_lag = temporal_lag * masks[:, t, 0].unsqueeze(1) + 1
                prediction, hidden = model(current_input, hidden, temporal_lag)
                current_input = (prediction * (1 - masks[:, t, :]) + inputs[:, t, :] * masks[:, t, :]).unsqueeze(1)
                loss += criterion(prediction - inputs[:, t-1, :], inputs[:, t, :]-inputs[:, t-1, :]) #(current_input-previous_input).squeeze(1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')


def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(0.1)
    if isinstance(m, nn.GRU):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0.1)  # Often zero initialized or could use a small constant.

train_mask = generate_masks(train_trajectories_tensor, missing_ratio=0.5, complete_traj_ratio=0.4)
train_dataset = myDataset(train_trajectories_tensor, train_mask)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

model = GRU_td(input_size=2, hidden_size=128, num_layers=2, num_classes=2).to(device)
model.apply(init_weights)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 50, optimizer, criterion)

Epoch 1, Loss: 1429690.29375
Epoch 2, Loss: 1044071.85
Epoch 3, Loss: 627357.88125
Epoch 4, Loss: 383743.1640625
Epoch 5, Loss: 299965.41953125
Epoch 6, Loss: 296803.60546875
Epoch 7, Loss: 296986.98671875
Epoch 8, Loss: 292401.2015625
Epoch 9, Loss: 287203.9546875
Epoch 10, Loss: 288235.2421875
Epoch 11, Loss: 264085.63515625
Epoch 12, Loss: 331606.371875
Epoch 13, Loss: 356302.04453125
Epoch 14, Loss: 317627.4140625
Epoch 15, Loss: 300824.078125
Epoch 16, Loss: 296312.79453125
Epoch 17, Loss: 259368.5875
Epoch 18, Loss: 228458.4671875
Epoch 19, Loss: 220579.4640625
Epoch 20, Loss: 226139.85078125
Epoch 21, Loss: 225675.0296875
Epoch 22, Loss: 217279.6421875
Epoch 23, Loss: 214742.98046875
Epoch 24, Loss: 213045.0421875
Epoch 25, Loss: 215873.23671875
Epoch 26, Loss: 215575.22890625
Epoch 27, Loss: 228614.2140625
Epoch 28, Loss: 204927.2234375
Epoch 29, Loss: 208276.9640625
Epoch 30, Loss: 198247.57265625
Epoch 31, Loss: 202366.47421875
Epoch 32, Loss: 206767.88671875
Epoch 33, Loss: 

In [153]:
def checkModel_td(model, inputs, masks):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(seq_len):
            if t <= 1:
                temporal_lag = torch.ones(inputs.size(0), 1).to(device)
            else:
                temporal_lag = temporal_lag * masks[:, t, 0].unsqueeze(1) + 1
            prediction, hidden = model(current_input, hidden, temporal_lag)
            current_input = (prediction * (masks[:, t, :]) + inputs[:, t, :] * (1 - masks[:, t, :])).unsqueeze(1)
            print(prediction[1], inputs[:, t, :][1])

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=0.9, complete_traj_ratio=0)
val_dataset = myDataset(val_trajectories_tensor, val_mask)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
inputs, masks = next(iter(val_dataloader))
checkModel_td(model, inputs, masks)

tensor([85.2072, 56.3504], device='cuda:0') tensor([92.5594, 68.6728], device='cuda:0')
tensor([93.2192, 61.9691], device='cuda:0') tensor([92.7287, 68.4787], device='cuda:0')
tensor([91.9524, 59.9048], device='cuda:0') tensor([93.0466, 68.1145], device='cuda:0')
tensor([96.0634, 58.1660], device='cuda:0') tensor([93.5100, 67.5835], device='cuda:0')
tensor([99.6730, 57.2302], device='cuda:0') tensor([93.9512, 67.0780], device='cuda:0')
tensor([102.8844,  56.6360], device='cuda:0') tensor([94.4516, 66.5048], device='cuda:0')
tensor([106.1619,  56.1906], device='cuda:0') tensor([94.8960, 65.9956], device='cuda:0')
tensor([110.0140,  55.6581], device='cuda:0') tensor([95.4006, 65.4149], device='cuda:0')
tensor([115.2693,  54.8968], device='cuda:0') tensor([95.8295, 64.9147], device='cuda:0')
tensor([122.2969,  54.0681], device='cuda:0') tensor([96.2558, 64.4177], device='cuda:0')
tensor([128.3391,  53.4559], device='cuda:0') tensor([96.7605, 63.8326], device='cuda:0')
tensor([130.3291,  5

In [154]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            if t <= 1:
                temporal_lag = torch.ones(inputs.size(0), 1).to(device)
            else:
                temporal_lag = temporal_lag * masks[:, t, 0].unsqueeze(1) + 1
            prediction, hidden = model(current_input, hidden, temporal_lag)
            current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((prediction - inputs[:, t, :]) * 10, dim=1) * (inputs[:, t, :] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

for i in [0.9, 0.8, 0.7, 0.6, 0.5]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = myDataset(val_trajectories_tensor, val_mask)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 354.1499569905686
Validation Loss: 330.9949540720715
Validation Loss: 315.035002764892
Validation Loss: 302.088465024116
Validation Loss: 289.88019185278483
